### BaseLearner8 - Test Reporducibility

In [1]:
from fastai2.vision.all import *
import sys, torch
sys.path.append('..')

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

from modules.expbuilder import default_params, run_exp, rm_test_recorder

train_path = Path('../../../rf-chess-data/cropped_v1/')
train_clean_path = Path('../../../rf-chess-data/cropped_v1.1/')

test_path_all = Path('../../../other-chess-data/regulation-test-2-all/')
test_path_easy = Path('../../../other-chess-data/regulation-test-2-easy/')

In [2]:
my_params = {'_train_seed':0, '_custom_crop': 'my-top-crop', 
             '_model_arch': resnet50,
            '_train_path':train_clean_path,
            '_learn_norm': False,
            }
default_params.update(my_params)

In [3]:
set_seed(42)
train_dl, learn = run_exp(default_params, 'dummy', b_ret=True)

##### INITIAL FIT

In [4]:
with learn.no_logging(): learn.fit_one_cycle(10)

In [5]:
pd.DataFrame(learn.cbs[3].values).iloc[-2:,:]

,0,1,2,3
8,0.983053,0.732456,0.942982,0.785088
9,0.906312,0.750000,0.947368,0.793860


##### SAVE IT

In [6]:
learn.path = Path('../')
learn.save('base-learner-8')

In [7]:
!ls ../models/base-learner-8*

../models/base-learner-8.pth


##### NEW LOADED MODEL

In [8]:
set_seed(42)
train_dl_2, learn_2 = run_exp(default_params, 'dummy', b_ret=True)

In [9]:
learn_2.path = Path('../')

In [10]:
learn_2.cbs

(#4) [TrainEvalCallback,Recorder,ProgressCallback,TestSetRecorder]

In [11]:
learn_2.load('base-learner-8')

##### TEST FOR EQUIVALENCE

In [23]:
test_eq(
    learn_2.model.state_dict().values(),
    learn.model.state_dict().values()
)

test_eq(
    learn_2.model.state_dict().keys(),
    learn.model.state_dict().keys()
)

In [28]:
test_eq( learn.opt.state_dict().keys(),
        learn_2.opt.state_dict().keys()
       )
# test_eq( learn.opt.state_dict().values(),
#          learn_2.opt.state_dict().values()
#        )

In [31]:
len(learn.opt.state_dict()['state'])

165

In [32]:
len(learn_2.opt.state_dict()['state'])

165

##### DIFFERENT RUNS

In [12]:
set_seed(20)
with learn.no_logging(): learn.fine_tune(20)

In [13]:
pd.DataFrame(learn.cbs[3].values).iloc[-2:,:]

,0,1,2,3
29,0.478619,0.855263,0.973684,0.881579
30,0.435804,0.868421,0.978070,0.894737


In [14]:
set_seed(20)
with learn_2.no_logging(): learn_2.fine_tune(20)

In [15]:
pd.DataFrame(learn_2.cbs[3].values).iloc[-2:,:]

,0,1,2,3
19,0.493076,0.824561,0.978070,0.850877
20,0.439624,0.842105,0.982456,0.872807


So, clearly they don't work the exact same way